In [48]:
# coding: utf-8

# # Dynet segmentation with tf fold


#just a bunch of fun
import numpy as np
import six
import time
import time
import data
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td
from conv_lstm_cell import *

# params
EMBEDDING_SIZE = 64
SEP = "|"
BATCH_SIZE = 100
data_dir = "/mnt/permanent/Home/nessie/velkey/data/"

#our alphabet

vocabulary = data.vocabulary(data_dir + 'vocabulary')
vsize=len(vocabulary)
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]




class data():
    def __init__(self, folder, truncate=500):
        self.data_dir = folder
        self.data = dict()
        self.size = dict()
        self.datasets = ["train", "test", "validation"]
        self.truncate = truncate
        
        for dataset in self.datasets:
            self.data[dataset] = self.sentence_reader(folder+dataset)
            #self.size[dataset] = sum(1 for line in open(folder+dataset))

                        
    def sentence_reader(self, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        data = [line[:-1].split('\t') for line in open(file)]
        while True:
            for item in data:
                print(item)
                tags = [int(num) for num in item[1]]
                if len(item[0]) == len(tags) and len(tags) != 0:
                    sent_onehot = self.onehot(item[0])
                    if len(sent_onehot) >= self.truncate:
                        sent_onehot=sent_onehot[:self.truncate]
                        tags = tags[:self.truncate]
                    yield (sent_onehot, tags)    
          
            
    def onehot(self, string):
        onehot = np.zeros([len(string),vsize])
        indices = np.arange(len(string)), np.array([int(index(char)) for char in string])
        onehot[indices]=1
        return [onehot[i,:] for i in range(len(onehot))]
            
store = data("/mnt/permanent/Home/nessie/velkey/data/")

# ## helper functions

def params_info():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable.name, shape)
        # print(len(shape))
        variable_parametes = 1
        for dim in shape:
            # print(dim)
            variable_parametes *= dim.value
        print("\tparams: ", variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)
    return total_parameters

def onehot(string):
    onehot = np.zeros([len(string),vsize])
    onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
    return [onehot[i,:] for i in range(len(onehot))]

def convLSTM_cell(kernel_size, out_features = 64):
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=out_features, kernel_shape=[kernel_size])
    return td.ScopedLayer(convlstm)

def multi_convLSTM_cell(kernel_sizes, out_features):
    stacked_convLSTM = tf.contrib.rnn.MultiRNNCell()
    return td.ScopedLayer(stacked_convLSTM)

def FC_cell(units):
    return td.ScopedLayer(tf.contrib.rnn.LSTMCell(num_units=units))

def multi_FC_cell(units_list):
    return td.ScopedLayer(tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(num_units=units) for units in units_list]))
    
def bidirectional_dynamic_CONV(fw_cell, bw_cell, out_features=64):
    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(fw_cell) >> td.GetItem(0)).reads(fw_seq)
        back_dir = (td.RNN(bw_cell) >> td.GetItem(0)).reads(bw_seq)
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = (td.Function(lambda x: tf.reshape(x, [-1,vsize*out_features])) >>
                            #td.FC(10, activation=tf.nn.tanh) >>
                            td.FC(1, activation=None))
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> 
                                  td.Metric('logits'))).reads(forward_dir, back_to_leftright)
                    
        #tag_logits = td.Map(output_transform).reads(bidir_common)

        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm

def bidirectional_dynamic_FC(fw_cell, bw_cell, hidden):
    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(fw_cell) >> td.GetItem(0)).reads(fw_seq)
        back_dir = (td.RNN(bw_cell) >> td.GetItem(0)).reads(bw_seq)
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = td.FC(1, activation=tf.nn.sigmoid)
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> 
                                  td.Metric('logits'))).reads(forward_dir, back_to_leftright)

        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm


CONV_data = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize,1])))
CONV_model =  CONV_data >> bidirectional_dynamic_CONV(convLSTM_cell(vsize), convLSTM_cell(vsize)) >> td.Void()
#labels = td.Map(td.Scalar() >> td.Metric("labels")) >> td.Void()

FC_data = td.Map(td.Vector(vsize))#>> td.Function(lambda x: tf.reshape(x, [-1,vsize])))
FC_model = FC_data >> bidirectional_dynamic_FC(multi_FC_cell([100]), multi_FC_cell([100]),100) >>td.Void()






[' ', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '§', '°', 'Á', 'É', 'Í', 'Ó', 'Ö', 'Ú', 'Ü', 'á', 'ä', 'é', 'ë', 'í', 'ó', 'ö', 'ú', 'ü', 'Ő', 'ő', 'ű']


In [24]:
def bidirectional_dynamic_FC(fw_cell, bw_cell, hidden):
    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input[0])
        labels = (td.GetItem(1)>>td.Map(td.Metric("labels"))>>td.Void()).reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(fw_cell) >> td.GetItem(0)).reads(fw_seq)
        back_dir = (td.RNN(bw_cell) >> td.GetItem(0)).reads(bw_seq)
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = td.FC(1, activation=tf.nn.sigmoid)
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> td.Metric('logits'))).reads(forward_dir, back_to_leftright)
        
        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm

d = td.Record((td.Map(td.Vector(vsize)),td.Map(td.Scalar())))
f = d >> bidirectional_dynamic_FC(multi_FC_cell([100]), multi_FC_cell([100]),100) >>td.Void()



In [25]:
compiler = td.Compiler.create(f)
logits = tf.squeeze(compiler.metric_tensors['logits'])
labels = compiler.metric_tensors['labels']

loss = tf.reduce_mean(tf.abs(tf.subtract(labels,logits)))
l2_loss = tf.reduce_mean(tf.abs(tf.subtract(labels,logits)))
log_loss = (labels) * tf.log(logits) + (1 - labels) * tf.log(1 - logits)
#cross_entropy_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels)
best_loss =  labels * -tf.log(logits) + (1 - labels) * -tf.log(1 - logits)
#TODO data label distribution analysis for determining the better best lost ;)


opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = opt.minimize(best_loss)
sess.run(tf.global_variables_initializer())


/home/velkey/Envs/tf/lib/python3.4/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [32]:
sess.run(tf.global_variables_initializer())
#feed = compiler.build_feed_dict([(onehot("naGon jó ötlet"),[0,0,0,1,0,1,0,0,0,0,0,1,0,0]) for i in range(1)])C

print(x)
feed = compiler.build_feed_dict([x for _ in range(BATCH_SIZE)])
for i in range(100):
    a,b,c,d= sess.run([logits, compiler.metric_tensors['labels'], best_loss, train_op], compiler.build_feed_dict([next(store.data["train"]) for _ in range(BATCH_SIZE)]))
    print("step: ", i)
    print("preds: ", a)
    print("labels: ", b)
    print("loss: ", c, '\n')
    accuracy=a > 0.5
    print("accuracy: ", np.sum(np.equal(accuracy,b))/len(accuracy)*100, "%")



([array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.])], [0])
step:  0
preds:  [ 0.50025189  0.50037885  0.49973455  0.49970192  0.49934009  0.49970192
  0.50037885  0.49922657  0.50037885  0.50050235  0.50037885  0.50025189
  0.50037885  0.49973455  0.49970192  0.49934009  0.49970192  0.50037885
  0.49922657  0.50037885  0.50050235  0.50037885  0.50025189  0.50037885
  0.49973455  0.49970192  0.49934009  0.49970192  0.50037885  0.49922657
  0.50037885  0.50050235  0.50037885  0.50025189  0.50037

step:  3
preds:  [ 0.49769706  0.49771157  0.49769706  0.49746576  0.49742925  0.49746576
  0.49892324  0.49746576  0.49851859  0.49746576  0.49805489  0.49769706
  0.49771157  0.49769706  0.49746576  0.49742925  0.49746576  0.49892324
  0.49746576  0.49851859  0.49746576  0.49805489  0.49769706  0.49771157
  0.49769706  0.49746576  0.49742925  0.49746576  0.49892324  0.49746576
  0.49851859  0.49746576  0.49805489  0.49769706  0.49771157  0.49769706
  0.49746576  0.49742925  0.49746576  0.49892324  0.49746576  0.49851859
  0.49746576  0.49805489  0.49769706  0.49771157  0.49769706  0.49746576
  0.49742925  0.49746576  0.49892324  0.49746576  0.49851859  0.49746576
  0.49805489  0.49769706  0.49771157  0.49769706  0.49746576  0.49742925
  0.49746576  0.49892324  0.49746576  0.49851859  0.49746576  0.49805489
  0.49769706  0.49771157  0.49769706  0.49746576  0.49742925  0.49746576
  0.49892324  0.49746576  0.49851859  0.49746576  0.49805489  0.49769706
  0.49771157  0.49769706  0.497465

step:  6
preds:  [ 0.49446636  0.49551037  0.49446636  0.49716952  0.49446636  0.49666452
  0.49446636  0.4962641   0.49554339  0.49595067  0.49554339  0.49446636
  0.49551037  0.49446636  0.49716952  0.49446636  0.49666452  0.49446636
  0.4962641   0.49554339  0.49595067  0.49554339  0.49446636  0.49551037
  0.49446636  0.49716952  0.49446636  0.49666452  0.49446636  0.4962641
  0.49554339  0.49595067  0.49554339  0.49446636  0.49551037  0.49446636
  0.49716952  0.49446636  0.49666452  0.49446636  0.4962641   0.49554339
  0.49595067  0.49554339  0.49446636  0.49551037  0.49446636  0.49716952
  0.49446636  0.49666452  0.49446636  0.4962641   0.49554339  0.49595067
  0.49554339  0.49446636  0.49551037  0.49446636  0.49716952  0.49446636
  0.49666452  0.49446636  0.4962641   0.49554339  0.49595067  0.49554339
  0.49446636  0.49551037  0.49446636  0.49716952  0.49446636  0.49666452
  0.49446636  0.4962641   0.49554339  0.49595067  0.49554339  0.49446636
  0.49551037  0.49446636  0.4971695

step:  9
preds:  [ 0.49516043  0.49126688  0.49456781  0.49126688  0.49423242  0.49313992
  0.49393752  0.49313992  0.49126688  0.49334145  0.49126688  0.49516043
  0.49126688  0.49456781  0.49126688  0.49423242  0.49313992  0.49393752
  0.49313992  0.49126688  0.49334145  0.49126688  0.49516043  0.49126688
  0.49456781  0.49126688  0.49423242  0.49313992  0.49393752  0.49313992
  0.49126688  0.49334145  0.49126688  0.49516043  0.49126688  0.49456781
  0.49126688  0.49423242  0.49313992  0.49393752  0.49313992  0.49126688
  0.49334145  0.49126688  0.49516043  0.49126688  0.49456781  0.49126688
  0.49423242  0.49313992  0.49393752  0.49313992  0.49126688  0.49334145
  0.49126688  0.49516043  0.49126688  0.49456781  0.49126688  0.49423242
  0.49313992  0.49393752  0.49313992  0.49126688  0.49334145  0.49126688
  0.49516043  0.49126688  0.49456781  0.49126688  0.49423242  0.49313992
  0.49393752  0.49313992  0.49126688  0.49334145  0.49126688  0.49516043
  0.49126688  0.49456781  0.491266

step:  12
preds:  [ 0.48776796  0.49184123  0.49036321  0.49154761  0.49036321  0.48776796
  0.49081692  0.48776796  0.49281013  0.48776796  0.49212378  0.48776796
  0.49184123  0.49036321  0.49154761  0.49036321  0.48776796  0.49081692
  0.48776796  0.49281013  0.48776796  0.49212378  0.48776796  0.49184123
  0.49036321  0.49154761  0.49036321  0.48776796  0.49081692  0.48776796
  0.49281013  0.48776796  0.49212378  0.48776796  0.49184123  0.49036321
  0.49154761  0.49036321  0.48776796  0.49081692  0.48776796  0.49281013
  0.48776796  0.49212378  0.48776796  0.49184123  0.49036321  0.49154761
  0.49036321  0.48776796  0.49081692  0.48776796  0.49281013  0.48776796
  0.49212378  0.48776796  0.49184123  0.49036321  0.49154761  0.49036321
  0.48776796  0.49081692  0.48776796  0.49281013  0.48776796  0.49212378
  0.48776796  0.49184123  0.49036321  0.49154761  0.49036321  0.48776796
  0.49081692  0.48776796  0.49281013  0.48776796  0.49212378  0.48776796
  0.49184123  0.49036321  0.49154

...

step:  60
preds:  [ 0.34106898  0.32840902  0.34648237  0.32840902  0.35345817  0.32840902
  0.35017917  0.32840902  0.34863231  0.34106898  0.34821007  0.34106898
  0.32840902  0.34648237  0.32840902  0.35345817  0.32840902  0.35017917
  0.32840902  0.34863231  0.34106898  0.34821007  0.34106898  0.32840902
  0.34648237  0.32840902  0.35345817  0.32840902  0.35017917  0.32840902
  0.34863231  0.34106898  0.34821007  0.34106898  0.32840902  0.34648237
  0.32840902  0.35345817  0.32840902  0.35017917  0.32840902  0.34863231
  0.34106898  0.34821007  0.34106898  0.32840902  0.34648237  0.32840902
  0.35345817  0.32840902  0.35017917  0.32840902  0.34863231  0.34106898
  0.34821007  0.34106898  0.32840902  0.34648237  0.32840902  0.35345817
  0.32840902  0.35017917  0.32840902  0.34863231  0.34106898  0.34821007
  0.34106898  0.32840902  0.34648237  0.32840902  0.35345817  0.32840902
  0.35017917  0.32840902  0.34863231  0.34106898  0.34821007  0.34106898
  0.32840902  0.34648237  0.32840

step:  63
preds:  [ 0.31089392  0.33692712  0.31089392  0.3335495   0.31089392  0.33182395
  0.32402083  0.33152488  0.32402083  0.31089392  0.32970753  0.31089392
  0.33692712  0.31089392  0.3335495   0.31089392  0.33182395  0.32402083
  0.33152488  0.32402083  0.31089392  0.32970753  0.31089392  0.33692712
  0.31089392  0.3335495   0.31089392  0.33182395  0.32402083  0.33152488
  0.32402083  0.31089392  0.32970753  0.31089392  0.33692712  0.31089392
  0.3335495   0.31089392  0.33182395  0.32402083  0.33152488  0.32402083
  0.31089392  0.32970753  0.31089392  0.33692712  0.31089392  0.3335495
  0.31089392  0.33182395  0.32402083  0.33152488  0.32402083  0.31089392
  0.32970753  0.31089392  0.33692712  0.31089392  0.3335495   0.31089392
  0.33182395  0.32402083  0.33152488  0.32402083  0.31089392  0.32970753
  0.31089392  0.33692712  0.31089392  0.3335495   0.31089392  0.33182395
  0.32402083  0.33152488  0.32402083  0.31089392  0.32970753  0.31089392
  0.33692712  0.31089392  0.333549

step:  66
preds:  [ 0.31630692  0.29290819  0.3144123   0.30642524  0.31424978  0.30642524
  0.29290819  0.31232756  0.29290819  0.31973588  0.29290819  0.31630692
  0.29290819  0.3144123   0.30642524  0.31424978  0.30642524  0.29290819
  0.31232756  0.29290819  0.31973588  0.29290819  0.31630692  0.29290819
  0.3144123   0.30642524  0.31424978  0.30642524  0.29290819  0.31232756
  0.29290819  0.31973588  0.29290819  0.31630692  0.29290819  0.3144123
  0.30642524  0.31424978  0.30642524  0.29290819  0.31232756  0.29290819
  0.31973588  0.29290819  0.31630692  0.29290819  0.3144123   0.30642524
  0.31424978  0.30642524  0.29290819  0.31232756  0.29290819  0.31973588
  0.29290819  0.31630692  0.29290819  0.3144123   0.30642524  0.31424978
  0.30642524  0.29290819  0.31232756  0.29290819  0.31973588  0.29290819
  0.31630692  0.29290819  0.3144123   0.30642524  0.31424978  0.30642524
  0.29290819  0.31232756  0.29290819  0.31973588  0.29290819  0.31630692
  0.29290819  0.3144123   0.306425

step:  69
preds:  [ 0.28845042  0.29654291  0.28845042  0.27464116  0.29452929  0.27464116
  0.30206236  0.27464116  0.2985791   0.27464116  0.29654768  0.28845042
  0.29654291  0.28845042  0.27464116  0.29452929  0.27464116  0.30206236
  0.27464116  0.2985791   0.27464116  0.29654768  0.28845042  0.29654291
  0.28845042  0.27464116  0.29452929  0.27464116  0.30206236  0.27464116
  0.2985791   0.27464116  0.29654768  0.28845042  0.29654291  0.28845042
  0.27464116  0.29452929  0.27464116  0.30206236  0.27464116  0.2985791
  0.27464116  0.29654768  0.28845042  0.29654291  0.28845042  0.27464116
  0.29452929  0.27464116  0.30206236  0.27464116  0.2985791   0.27464116
  0.29654768  0.28845042  0.29654291  0.28845042  0.27464116  0.29452929
  0.27464116  0.30206236  0.27464116  0.2985791   0.27464116  0.29654768
  0.28845042  0.29654291  0.28845042  0.27464116  0.29452929  0.27464116
  0.30206236  0.27464116  0.2985791   0.27464116  0.29654768  0.28845042
  0.29654291  0.28845042  0.274641

KeyboardInterrupt: 